# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
orders = pd.read_csv('Orders.csv')

In [4]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    397924 non-null  int64  
 1   InvoiceNo     397924 non-null  int64  
 2   StockCode     397924 non-null  object 
 3   year          397924 non-null  int64  
 4   month         397924 non-null  int64  
 5   day           397924 non-null  int64  
 6   hour          397924 non-null  int64  
 7   Description   397924 non-null  object 
 8   Quantity      397924 non-null  int64  
 9   InvoiceDate   397924 non-null  object 
 10  UnitPrice     397924 non-null  float64
 11  CustomerID    397924 non-null  int64  
 12  Country       397924 non-null  object 
 13  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


In [5]:
orders.describe()

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,CustomerID,amount_spent
count,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000
mean,278465.221859,560617.126645,2010.934259,7.612537,3.614555,12.728247,13.021823,3.116174,15294.315171,22.394749
std,152771.368303,13106.167695,0.247829,3.416527,1.928274,2.273535,180.420210,22.096788,1713.169877,309.055588
min,0.000000,536365.000000,2010.000000,1.000000,1.000000,6.000000,1.000000,0.000000,12346.000000,0.000000
25%,148333.750000,549234.000000,2011.000000,5.000000,2.000000,11.000000,2.000000,1.250000,13969.000000,4.680000
50%,284907.500000,561893.000000,2011.000000,8.000000,3.000000,13.000000,6.000000,1.950000,15159.000000,11.800000
75%,410079.250000,572090.000000,2011.000000,11.000000,5.000000,14.000000,12.000000,3.750000,16795.000000,19.800000
max,541908.000000,581587.000000,2011.000000,12.000000,7.000000,20.000000,80995.000000,8142.750000,18287.000000,168469.600000


In [6]:
orders.shape

(397924, 14)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [10]:
# your code here
x = orders.groupby('CustomerID').agg({'amount_spent': 'sum'})
x

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [19]:
x['VIP'] = x['amount_spent']> x['amount_spent'].quantile(0.95)

In [21]:
x['Preferred'] = (x['amount_spent']> x['amount_spent'].quantile(0.75))& (x['amount_spent']<x['amount_spent'].quantile(0.95))

In [22]:
x

,amount_spent,VIP,Preferred
CustomerID,,,
12346,77183.60,True,False
12347,4310.00,False,True
12348,1797.24,False,True
12349,1757.55,False,True
12350,334.40,False,False
...,...,...,...
18280,180.60,False,False
18281,80.82,False,False
18282,178.05,False,False


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [28]:
# your code here
y = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent': 'sum'})
y

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40
...,...,...
18280,United Kingdom,180.60
18281,United Kingdom,80.82
18282,United Kingdom,178.05


In [29]:
y['VIP'] = y['amount_spent']> y['amount_spent'].quantile(0.95)

In [30]:
y['Preferred'] = (y['amount_spent']> y['amount_spent'].quantile(0.75))& (y['amount_spent']<y['amount_spent'].quantile(0.95))

In [31]:
y

,,amount_spent,VIP,Preferred
CustomerID,Country,,,
12346,United Kingdom,77183.60,True,False
12347,Iceland,4310.00,False,True
12348,Finland,1797.24,False,True
12349,Italy,1757.55,False,True
12350,Norway,334.40,False,False
...,...,...,...,...
18280,United Kingdom,180.60,False,False
18281,United Kingdom,80.82,False,False
18282,United Kingdom,178.05,False,False


In [36]:
z = y.groupby('Country').agg({'VIP':'count'})

In [39]:
z.sort_values(by='VIP', ascending =False)

,VIP
Country,
United Kingdom,3921
Germany,94
France,87
Spain,30
Belgium,25
Switzerland,21
Portugal,19
Italy,14
Finland,12


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here